 Alpine is a lightweight Linux distribution so it is quick to pull down and run, making it a popular starting point for many other images.

# Docker Container-Centric Commands for Beginners:
(https://www.alibabacloud.com/blog/594562)

docker container commit - Create a new image from a container's changes
docker container export - Export a container's filesystem as a tar archive

These 2 commands have different uses:

commit: create a new image. You can then build a new container using this image as the base image. The new container is like a clone of original container. environment variables still exist.
export: save container content as a tar archive file. You can then untar and investigate the file content at any Linux shell. Original container totally gone. All we have is a file system: no environment variables and no concept of 'container running state'.

In [2]:
!ls
%cd ../../media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/nano/

Desktop    Downloads  localperl  Pictures  snap       Videos
Documents  gopath     Music	 Public    Templates
/media/hossein/HDD/gdrive/SkillsLearningTraining/docker-tutorials/main/nano


add this bash file in directory nano
```
remove-me#!/bin/bash
function SIGINT_trap() {
  echo . . . SIGINT signal caught
  echo . . . doing SIGINT cleanup
  exit
}

function SIGTERM_trap() {
  echo . . . SIGTERM signal caught
  echo . . . doing SIGTERM cleanup
  exit
}

trap SIGINT_trap SIGINT
trap SIGTERM_trap SIGTERM

for i in `seq 1 50`; do
    sleep 1
    echo -n " . "
    echo $i
done

exit 0
```

add the following ines in the Dockerfile
```
FROM alpine:3.8
ENV myvar original value
COPY traps /root/
RUN chmod +x /root/traps
CMD ["/bin/sh", "/root/traps"]
```

In [3]:
!docker build --tag mytraps:demo --file Dockerfile  .


Sending build context to Docker daemon  3.072kB
Step 1/5 : FROM alpine:3.8
3.8: Pulling from library/alpine

Digest: sha256:2bb501e6173d9d006e56de5bce2720eb06396803300fe1687b58a7ff32bf4c14
Status: Downloaded newer image for alpine:3.8
 ---> c8bccc0af957
Step 2/5 : ENV myvar original value
 ---> Running in 09b02ace3c57
Removing intermediate container 09b02ace3c57
 ---> b1b0f2fddea5
Step 3/5 : COPY traps /root/
COPY failed: stat /var/lib/docker/tmp/docker-builder242008870/traps: no such file or directory


well this failed since I had added the .sh at the end of the name, so let's change it

In [4]:
!docker build --tag mytraps:demo --file Dockerfile  .


Sending build context to Docker daemon  3.072kB
Step 1/5 : FROM alpine:3.8
 ---> c8bccc0af957
Step 2/5 : ENV myvar original value
 ---> Using cache
 ---> b1b0f2fddea5
Step 3/5 : COPY traps /root/
 ---> 375322053203
Step 4/5 : RUN chmod +x /root/traps
 ---> Running in 9501121f99a1
Removing intermediate container 9501121f99a1
 ---> 6fb4d9ec0797
Step 5/5 : CMD ["/bin/sh", "/root/traps"]
 ---> Running in 57cca59cdbe8
Removing intermediate container 57cca59cdbe8
 ---> 3947504706bc
Successfully built 3947504706bc
Successfully tagged mytraps:demo


In [11]:
!docker container run -d --name mycontain mytraps:demo


86c3c3e95cf585678020bae332b79e8ec1a1f4780e1fb03b2931e504abf9f6da


In [12]:
!docker container logs mycontain -f

 . 1
 . 2
 . 3
 . 4
 . 5
 . 6
 . 7
 . 8
 . 9
 . 10
 . 11
 . 12
 . 13
 . 14
 . 15
 . 16
 . 17
 . 18
 . 19
 . 20
 . 21
 . 22
 . 23
 . 24
 . 25
 . 26
 . 27
 . 28
 . 29
 . 30
 . 31
 . 32
 . 33
 . 34
 . 35
 . 36
 . 37
 . 38
 . 39
 . 40
 . 41
 . 42
 . 43
 . 44
 . 45
 . 46
 . 47
 . 48
 . 49
 . 50


ok then remove the container and run it again

In [13]:
!docker rm mycontain

mycontain


then run it again and this time send the second commad while running!

In [17]:
!docker container run -d --name mycontain mytraps:demo && docker container logs mycontain -f


3cc36ebcce0a1408f12bbb173e41f4bd5b46a35ddceec5b7d5df46286b4c0f6b
 . 1
 . 2
 . 3
 . 4
 . 5
 . 6
 . 7
 . 8
 . 9
 . 10
 . 11
 . 12
 . 13
 . 14
 . 15
 . 16
 . 17
 . 18
 . 19
 . 20
 . 21
 . 22
 . 23
 . 24
 . 25
 . 26
 . 27
 . 28
 . 29
 . 30
 . 31
 . 32
 . 33
 . 34
 . 35
 . 36
 . 37
 . 38
 . 39
 . 40
 . 41
 . 42
 . 43
 . 44
 . 45
 . 46
 . 47
 . 48
 . 49
 . 50
Error response from daemon: Cannot kill container: mycontain: Container 3cc36ebcce0a1408f12bbb173e41f4bd5b46a35ddceec5b7d5df46286b4c0f6b is not running


in another terminal enter this line

In [15]:
!docker container kill --signal 15  mycontain


Error response from daemon: Cannot kill container: mycontain: Container e141bfeb675750293bc38866c0516869c8cf2678b0fd6b9fe5e9558441c7cbed is not running


summary: your processes running in your containers must have the proper cleanup routines. With Docker you will be shutting down and restart containers much more frequently than in the pre-Docker world. So its critical your containers can correctly handle all the different ways of being shut down.

Based on these exercises you see how easy it is to test handling just 2 signals.

If you enter trap -l at your shell you will see a list of 64 possible signals. Your processes must be able to handle all the possible values they can receive.

docker container kill is wrongly named.

It should have been: docker container signal - since you can send signals just to re-read configuration files. Not all signals kill.

#Docker for IT Pros and System Administrators Stage 1

https://training.play-with-docker.com/ops-stage1/

In [18]:
!docker run -d -p 30:80 --name myserver nginx

sh: 0: getcwd() failed: No such file or directory
Unable to find image 'nginx:latest' locally
latest: Pulling from library/nginx





Digest: sha256:fc66cdef5ca33809823182c9c5d72ea86fd2cef7713cf3363e1a0b12a5d77500
Status: Downloaded newer image for nginx:latest
531e654272af1052a125b6f764d5a34d7e1df1f219c7b274dcaa0c9a248a96a5
